<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/cifar100/CIFAR100_do_label_shift_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://zenodo.org/record/3458525/files/cifar100_pred_logits.txt.gz?download=1 -O cifar100_pred_logits.txt.gz
!wget https://zenodo.org/record/3458525/files/cifar100_ys_test.txt.gz?download=1 -O cifar100_ys_test.txt.gz


--2019-09-23 19:22:37--  https://zenodo.org/record/3458525/files/cifar100_pred_logits.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4717007 (4.5M) [application/octet-stream]
Saving to: ‘cifar100_pred_logits.txt.gz’

cifar100_pred_logit 100%[===================>]   4.50M  3.67MB/s    in 1.2s    

2019-09-23 19:22:39 (3.67 MB/s) - ‘cifar100_pred_logits.txt.gz’ saved [4717007/4717007]

--2019-09-23 19:22:40--  https://zenodo.org/record/3458525/files/cifar100_ys_test.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46058 (45K) [application/octet-stream]
Saving to: ‘cifar100_ys_test.txt.gz’

cifar100_ys_test.tx 100%[===================>]  44.98K   147KB/s    in 0.3s    

2019-09-23 19:22:41 (147 KB/s)

In [2]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

Cloning into 'abstention'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 330 (delta 72), reused 64 (delta 36), pack-reused 222
Receiving objects: 100% (330/330), 94.11 KiB | 9.41 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/abstention
Already up to date.
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=46220 sha256=8f52d0e38d95f01c4732642901cae2445c6db67bd445ce24307773875a9c9e54
  Stored in directory: /tmp/pip-ephem-wheel-cache-31i3c6k6/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [3]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

Cloning into 'labelshiftexperiments'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 13), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (32/32), done.
/content/labelshiftexperiments
Already up to date.
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-none-any.whl size=6273 sha256=ab7d3bbbcbb55f451f39d3c4e5b9607be045f346640ca3f42ede10b436ad0ffb
  Stored in directory: /tmp/pip-ephem-wheel-cache-stlv3gn1/wheels/6d/56/a7/fc736645f87832f3f9af94ef6661744069f41cd66bab35ec1b
Successfully built labelshiftexperiments
/content


In [0]:
!gunzip *.gz

In [8]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, VectorScaling, NoBiasVectorScaling, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
all_labels = cifarandmnist.read_preds(open("cifar100_ys_test.txt"))
all_predlogits = cifarandmnist.read_preds(open("cifar100_pred_logits.txt"))

print(np.mean(np.argmax(all_labels,axis=-1)
              ==np.argmax(all_predlogits, axis=-1)))

0.7048


In [9]:
len(all_labels)

10000

In [0]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, NoBiasVectorScaling, VectorScaling, CrossValidatedBCTS, softmax)
from abstention.label_shift import (EMImbalanceAdapter, BBSEImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('VS', VectorScaling(verbose=False)),
    ('NBVS', NoBiasVectorScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=np.arange(100))),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptname_to_adapter = OrderedDict([
    ('em', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True))
])
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'NBVS'),
    ('bbse-soft', 'BCTS'),
    ('bbse-soft', 'VS'),  
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'NBVS'),
    ('em', 'BCTS'),
    ('em', 'VS'),
    #('em', 'CVBCTS'),
]

In [40]:
from collections import defaultdict
import numpy as np
import sys
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import numpy as np
from collections import defaultdict, OrderedDict
import scipy

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist


num_trials = 100
alphas_and_samplesize = [(0., 5000)]

metric_to_adaptname_to_perfs = defaultdict(lambda: defaultdict(list))
alpha_to_samplesize_to_adaptername_to_metric_to_vals = (
        defaultdict(
            lambda: defaultdict(
                     lambda: defaultdict(
                              lambda: defaultdict(list)))))
alpha_to_samplesize_to_baselineperf = defaultdict(
    lambda: defaultdict(list))
metric_to_samplesize_to_calibname_to_unshiftedvals = defaultdict(
    lambda: defaultdict(lambda: defaultdict(list)))
samplesizesseen = set()

for (alpha, samplesize) in alphas_and_samplesize:
    for trial_num in range(num_trials):
        if (trial_num%1==0):
            print("Done",trial_num,"trials")
            sys.stdout.flush()
        rng = np.random.RandomState(trial_num*10)
        #sample a split of pseudovalid/pseudotest indices
        indices = rng.choice(np.arange(len(all_labels)),
                             size=len(all_labels),
                             replace=False)
        pseudovalid_indices = indices[:samplesize]
        pseudotest_indices = indices[samplesize:]
        assert ((len(pseudovalid_indices)+len(pseudotest_indices))
                ==len(all_labels))
        
        pseudovalid_preacts = all_predlogits[pseudovalid_indices]
        pseudotest_preacts = all_predlogits[pseudotest_indices]
        pseudovalid_labels = all_labels[pseudovalid_indices]
        pseudotest_labels = all_labels[pseudotest_indices]
        
        draw_test_indices = cifarandmnist.get_func_to_draw_label_proportions(
                              test_labels=pseudotest_labels)
        
        #perform calibration
        (calibname_to_calibfunc,
         calibname_to_calibpseudovalidpreds) = (
                    cifarandmnist.run_calibmethods(
                       valid_preacts=pseudovalid_preacts,
                       valid_labels=pseudovalid_labels,
                       test_preacts=pseudotest_preacts,
                       test_labels=pseudotest_labels,
                       calibname_to_calibfactory=
                        calibname_to_calibfactory,
                       samplesize=samplesize,
                       samplesizesseen=samplesizesseen,
                       metric_to_samplesize_to_calibname_to_unshiftedvals=
                        metric_to_samplesize_to_calibname_to_unshiftedvals))
        
        #perform the dirichlet shift
        altered_class_priors = rng.dirichlet([alpha for x in range(100)])        
        pseudotest_indices = draw_test_indices(
                          total_to_return=samplesize,                    
                          label_proportions=altered_class_priors,
                          rng=rng)
        shifted_pseudotest_labels = pseudotest_labels[pseudotest_indices]
        shifted_pseudotest_preacts = pseudotest_preacts[pseudotest_indices]
        
        calibname_to_calibshiftedtestpreds = {}
        for (calibname, calibfunc) in calibname_to_calibfunc.items():
            calibname_to_calibshiftedtestpreds[calibname] =(
                calibfunc(shifted_pseudotest_preacts))
        
        shifted_pseudotest_baseline_perf = np.mean(
            np.argmax(shifted_pseudotest_preacts, axis=-1)
            ==np.argmax(shifted_pseudotest_labels, axis=-1))
        alpha_to_samplesize_to_baselineperf[alpha][samplesize].append(
            shifted_pseudotest_baseline_perf)
        true_shifted_priors = np.mean(shifted_pseudotest_labels, axis=0)
        
        for adapter_name,calib_name in adaptncalib_pairs:
            calib_shifted_pseudotest_preds =\
              calibname_to_calibshiftedtestpreds[calib_name]
            calib_pseudovalid_preds =\
              calibname_to_calibpseudovalidpreds[calib_name]
            imbalance_adapter = adaptname_to_adapter[adapter_name]
            imbalance_adapter_func = imbalance_adapter(
                valid_labels=pseudovalid_labels,                       
                tofit_initial_posterior_probs=calib_shifted_pseudotest_preds, 
                valid_posterior_probs=calib_pseudovalid_preds)
            adapted_shifted_pseudotest_preds = imbalance_adapter_func(        
                        calib_shifted_pseudotest_preds)
            estim_shifted_priors = np.mean(adapted_shifted_pseudotest_preds,  
                                           axis=0)
            #compute metrics for quality of shift
            adapted_shifted_pseudotest_perf = np.mean(
              np.argmax(adapted_shifted_pseudotest_preds, axis=-1)
              ==np.argmax(shifted_pseudotest_labels, axis=-1))
            delta_from_baseline = (adapted_shifted_pseudotest_perf-
                                   shifted_pseudotest_baseline_perf)
            
            alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'jsdiv'].append(
                  scipy.spatial.distance.jensenshannon(
                      p=true_shifted_priors, q=estim_shifted_priors))
            alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                alpha][samplesize][adapter_name+":"+calib_name][
                'delta_perf'].append(delta_from_baseline)
        
    if (samplesize not in samplesizesseen):
        print("Calibration stats")
        for metric in ['ece', 'nll', 'jsdiv']:
            print("Metric",metric)
            for calibname in calibname_to_calibfactory:
                print(calibname, np.mean(                     
                       metric_to_samplesize_to_calibname_to_unshiftedvals[
                        metric][samplesize][calibname]))  
        samplesizesseen.add(samplesize)

    print("On alpha",alpha,"sample size", samplesize)
    for metric_name in ['delta_perf', 'jsdiv']: 
        print("Metric",metric_name)
        for adapter_name,calib_name in adaptncalib_pairs:
            adaptncalib_name = adapter_name+":"+calib_name
            n = len(alpha_to_samplesize_to_adaptername_to_metric_to_vals[
                  alpha][samplesize][adaptncalib_name][metric_name])

            print(adaptncalib_name, np.mean(
             alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name]), "+/-",
             (1.0/np.sqrt(n))*np.std(
             alpha_to_samplesize_to_adaptername_to_metric_to_vals[
               alpha][samplesize][adaptncalib_name][metric_name],
               ddof=1))           
            sys.stdout.flush()

Done 0 trials


KeyboardInterrupt: ignored

In [34]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "alpha_to_samplesize_to_baselineperf":
     alpha_to_samplesize_to_baselineperf,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['alpha_to_samplesize_to_adaptername_to_metric_to_vals']
alpha_to_samplesize_to_baselineperf =\
    loaded_dicts['alpha_to_samplesize_to_baselineperf']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [36]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS",
                         "VS":"VS", "NBVS": "NBVS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['5000'],
        calibnames_in_table=['None', 'TS', 'NBVS', 'BCTS', 'VS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="KaggleDR Calibration metric differences", label="kaggledrcalibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c | c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{1}{| c}{nll} & \multicolumn{1}{| c}{jsdiv} & \multicolumn{1}{| c}{ECE}\\
    \cline{2-4}
    & $n$=5000 & $n$=5000 & $n$=5000\\
    \hline
    None & 1.517; 4.0 & 0.055; 4.0 & 18.035; 4.0\\
    TS & 1.111; 2.8 & 0.051; 3.0 & 3.632; 2.6\\
    NBVS & \textbf{1.099; 0.8} & \textbf{0.042; 0.6} & \textbf{2.94; 0.1}\\
    BCTS & \textbf{1.096; 0.2} & \textbf{0.043; 1.1} & 3.46; 2.1\\
    VS & 1.105; 2.2 & \textbf{0.043; 1.3} & 3.107; 1.2\\
  \end{tabular}}
  \caption{KaggleDR Calibration metric differences}
  \label{tab:kaggledrcalibrationcomparison}
\end{table*}



In [38]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:NBVS', 'em:BCTS', 'em:VS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
           'bbse-soft:TS', 'bbse-soft:NBVS', 'bbse-soft:BCTS', 'bbse-soft:VS'])])
metric_to_largerisbetter = {'perf':True}
samplesizes_in_table = ['5000']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'NBVS': 'NBVS',
                           'BCTS': 'BCTS',
                           'VS': 'VS'}

alphas_in_table = ['5']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: JS Divergence, tweakone shift",
    label="kaggledrjsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_perf',
    largerisbetter=True,
    alphas_in_table=alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="KaggleDR Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="kaggledrdeltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{1}{| c}{$\alpha=5$}\\ 
    \cline{3-3}
    & & $n$=5000\\
    \hline
    \hline
    EM & None & \textbf{0.08; 2.4}\\
    EM & TS & 0.084; 3.7\\
    EM & NBVS & \textbf{0.076; 0.9}\\
    EM & BCTS & \textbf{0.076; 1.2}\\
    EM & VS & \textbf{0.077; 1.8}\\
    \hline
    \hline
    BBSE-hard & None & 0.095; 4.5\\
    BBSE-soft & None & \textbf{0.089; 2.1}\\
    BBSE-soft & TS & 0.091; 3.7\\
    BBSE-soft & NBVS & \textbf{0.085; 1.4}\\
    BBSE-soft & BCTS & \textbf{0.085; 1.6}\\
    BBSE-soft & VS & \textbf{0.085; 1.7}\\
  \end{tabular}}
  \caption{KaggleDR Metric: JS Divergence, tweakone shift}
  \label{tab:kaggledrjsdivtweakoneshift}
\end{table*}


\begin{table*}
\adjustbox{max width=\textwidth}{
  \c